In [1]:
import os

requirements_installed = False
max_retries = 3
retries = 0


def install_requirements():
    """Installs the requirements from requirements.txt file"""
    global requirements_installed
    if requirements_installed:
        print("Requirements already installed.")
        return

    print("Installing requirements...")
    install_status = os.system("pip install -r requirements.txt")
    if install_status == 0:
        print("Requirements installed successfully.")
        requirements_installed = True
    else:
        print("Failed to install requirements.")
        if retries < max_retries:
            print("Retrying...")
            retries += 1
            return install_requirements()
        exit(1)
    return

In [2]:
install_requirements()

Installing requirements...
Requirements installed successfully.


In [3]:
from dotenv import load_dotenv
import os


def setup_env():
    """Sets up the environment variables"""
    load_dotenv()

    GROQ_API_KEY = os.getenv("GROQ_API_KEY")

    if GROQ_API_KEY is None:
        print("Please set the GROQ_API_KEY environment variable.")
        exit(1)
    else:
        print("GROQ_API_KEY is set.")

In [4]:
setup_env()

GROQ_API_KEY is set.


In [5]:
import instructor
from groq import Groq
import traceback
from pydantic import BaseModel
from typing import Union

DEFAULT_MODEL = "llama-3.3-70b-versatile"

class LLMErrorResponse(BaseModel):
    error: str


def get_groq_client():
    """Returns an instance of the Groq class"""
    groq = Groq(api_key=os.getenv("GROQ_API_KEY"))
    client = instructor.from_groq(groq, mode=instructor.Mode.JSON)
    return client


def llm(
    prompt: str,
    response_model: BaseModel,
    system="You are a helpful AI assistant. The user will talk to you and its your job to provide detailed and clear responses.",
    model=DEFAULT_MODEL,
) -> Union[BaseModel, LLMErrorResponse]:
    """Calls LLM API with the given prompt. Defaults to llama-3.3-70b-versatile""",
    try:
        client = get_groq_client()

        messages = [
            {"role": "system", "content": system},
            {"role": "user", "content": prompt},
        ]

        response = client.chat.completions.create(
            messages=messages, model=model, response_model=response_model
        )
        return response
    except Exception as e:
        traceback.print_exc()
        return LLMErrorResponse(error=str(e))

In [6]:
from typing import List, Union, Any
from pydantic import BaseModel
from groq import Groq


class UserStory(BaseModel):
    """Represents a User Story"""
    title: str
    description: str
    acceptance_criteria: List[str]
    story_points: int

class UserStories(BaseModel):
    """Represents a collection of User Stories"""
    user_stories: List[UserStory]

class PRD(BaseModel):
    """Represents a Product Requirements Document (PRD)"""
    title: str
    overview: str
    problem_statement: str
    business_domains: List[str]
    proposed_solution: str
    target_audience: str
    features: List[str]
    constraints: List[str]
    assumptions: List[str]
    dependencies: List[str]
    user_personas: List[str]
    conclusion: str


class MiniPM:
    """An AI agent that acts as a Product Manager to help project managers create PRD and User Stories for their projects. """
    llm: Groq
    project_description: str

    def __init__(self, project_description: str):
        """Initializes the MiniPM AI agent with the project description. """
        self.project_description = project_description
        self.llm = get_groq_client()

    def get_prd(self, response_format = "markdown", augment = True) -> Union[PRD, LLMErrorResponse, Any]:
        """Gets the Product Requirements Document (PRD) for the project. """
        system = """
                You are MiniPM; a Product Manager AI agent. 
                You help project managers to create a Product Requirements Document (PRD) for their projects.
                """
        prompt = f"""
                Provide a Product Requirements Document (PRD) for the project with the following details:
                Details: {self.project_description}
                """
        response = llm(
            prompt=prompt,
            response_model=PRD,
            system=system
        )

        if augment:
            response = self._augment_prd(response)

        if response_format == "json":
            return response.model_dump_json()
        elif response_format == "markdown":
            response_dict = response.model_dump()
            keys = response_dict.keys()
            markdown = f"# Product Requirements Document (PRD) for {response.title}\n"
            for key in keys:
                title = key.title().replace("_", " ").replace("Prd", "PRD")
                if key == "title":
                    title = ""
                    continue
                markdown += f"## {title}\n"
                value = response_dict[key]
                if type(value) == list:
                    value_bullets = [f"- {item}" for item in value]
                    value = "\n".join(value_bullets)
                    markdown += f"{value}\n"
                else:
                    markdown += f"{response_dict[key]}\n"
            return markdown
        elif response_format == "dto":
            return response 
        raise ValueError("Invalid response_format. Use 'json', 'markdown' or 'dto'.")
    
    def _augment_prd(self, prd: PRD) -> PRD:
        """Augments the PRD with greater details."""
        prd_json = prd.model_dump()
        print("Augmenting PRD: ", prd_json)
        keys = prd_json.keys()
        for key in keys:
            if key == "title":
                continue
            value = prd_json[key]
            prompt = f"Provide more details about the {key} for the project."
            if type(value) == str:
                prompt += f" Current value: {value}"
                expanded_value = llm(prompt, response_model=str)
                prd_json[key] = expanded_value
            elif type(value) == list:
                expanded_values = []
                for item in value:
                    prompt += f" Current value: {item}"
                    expanded_item = llm(prompt, response_model=str)
                    expanded_values.append(expanded_item)
                prd_json[key] = expanded_values
        print("Augmented PRD: ", prd_json)
        return PRD(**prd_json)


    def get_user_stories(self) -> Union[UserStories, LLMErrorResponse]:
        """Gets the User Stories for the project"""
        system = """
                You are MiniPM; a Product Manager AI agent. 
                You help project managers to create User Stories for their projects.
                """
        prompt = f"""
                Provide User Stories for the project with the following details:
                Details: {self.project_description}
                PRD: {self.get_prd()}
                """
        response = llm(
            prompt=prompt,
            response_model=UserStories,
            system=system
        )
        return response

In [7]:
from IPython.display import Markdown


def render_output(markdown: str) -> None:
    """Renders the generated output file as markdown."""
    return Markdown(markdown)

def save_markdown_file(markdown: str, file_path: str) -> None:
    """Saves the generated markdown file to the specified path."""
    with open(file_path, "w") as file:
        file.write(markdown)

In [8]:
## Input Parameters: Change these values to get different PRD and User Stories
project_description = "A web application for tracking investigating behavioural change in aliens."
output_file = "alien_behavioural_change_prd.md"
augment = True

## Create an instance of MiniPM AI agent
minipm = MiniPM(project_description)

## Get the Product Requirements Document (PRD)
prd = minipm.get_prd(response_format="markdown", augment=augment)

save_markdown_file(prd, output_file)

## Render the PRD as markdown
render_output(prd)


Augmenting PRD:  {'title': 'Alien Behavioural Change Tracker', 'overview': 'A web application designed to track and investigate behavioural changes in aliens, providing insights into their social and cultural development.', 'problem_statement': 'Currently, there is a lack of understanding and data on alien behavioural patterns, making it challenging for researchers to study and predict their actions.', 'business_domains': ['Astrobiology', 'Extraterrestrial Research', 'Behavioural Science'], 'proposed_solution': 'Develop a user-friendly web application that enables researchers to collect, analyze, and visualize data on alien behavioural changes, facilitating a deeper understanding of their social and cultural evolution.', 'target_audience': 'Researchers, scientists, and astrobiologists studying alien life forms and behaviour', 'features': ['Data collection and management system', 'Behavioural pattern analysis and visualization tools', 'Machine learning algorithms for predictive modellin

# Product Requirements Document (PRD) for Alien Behavioural Change Tracker
## Overview
The project involves developing a comprehensive web application to monitor and analyze behavioural patterns in extraterrestrial life forms, offering a unique perspective on their societal structures, cultural evolution, and potential interactions with other celestial entities. This application aims to serve as a central hub for intergalactic research, facilitating collaboration among experts from various disciplines to better understand the complexities of alien behaviour and its implications for the universe.
## Problem Statement
The problem statement for the project revolves around the current lack of understanding and data on alien behavioural patterns, which poses significant challenges for researchers aiming to study and predict their actions. This knowledge gap hampers the development of effective strategies for potential interactions and necessitates the collection and analysis of comprehensive data on alien behaviour, social structures, communication methods, and technological capabilities. The project seeks to address this issue by devising innovative methods for observing, recording, and interpreting alien behaviour, ultimately contributing to a deeper understanding of their patterns and habits.
## Business Domains
- The business_domains for the project, with the current value being Astrobiology, can be expanded to include interdisciplinary fields such as the search for extraterrestrial life, the study of extremophiles, and the exploration of planetary habitability. This domain involves the convergence of biology, astronomy, geology, and atmospheric science to understand the origins, evolution, distribution, and future of life in the universe.
- The business domains for the project include Astrobiology and Extraterrestrial Research. Astrobiology involves the study of the origin, evolution, distribution, and future of life in the universe, while Extraterrestrial Research focuses on the search for life beyond Earth and the exploration of celestial bodies such as planets, moons, and asteroids.
- The project involves three key business domains: Astrobiology, Extraterrestrial Research, and Behavioural Science. Astrobiology focuses on the study of the origin, evolution, distribution, and future of life in the universe. Extraterrestrial Research explores the search for life beyond Earth, including the study of exoplanets, asteroids, and other celestial bodies. Behavioural Science examines the human behaviour and societal implications of space exploration and potential encounters with extraterrestrial life.
## Proposed Solution
The proposed solution involves developing a user-friendly web application with the following key features: (1) Data Collection Module: allows researchers to input and store data on alien behavioural changes, including observations, experiments, and surveys. (2) Data Analysis Module: enables researchers to analyze the collected data using statistical models and machine learning algorithms, providing insights into patterns and trends. (3) Data Visualization Module: offers a range of visualization tools, such as graphs, charts, and heat maps, to help researchers represent complex data in a clear and understandable format. (4) Collaboration Module: facilitates collaboration among researchers by enabling them to share data, analysis, and visualizations, and engage in discussions and debates. (5) Knowledge Base Module: provides a repository of existing research and knowledge on alien social and cultural evolution, allowing researchers to contextualize their findings and identify areas for further study.
## Target Audience
The target audience for the project primarily consists of researchers, scientists, and astrobiologists who are actively involved in the study of alien life forms and their behavior. This audience can be further broken down into subgroups, including planetary scientists, microbiologists, astrophysicists, and experts in the fields of exopaleontology and the search for extraterrestrial intelligence (SETI). These individuals are likely to be affiliated with academic institutions, research centers, or space agencies, and are motivated by a desire to understand the origins of life in the universe, the potential for life on other planets, and the implications of discovering extraterrestrial life for our understanding of the cosmos and human existence.
## Features
- The data collection and management system is designed to efficiently gather, store, and analyze data from various sources. Key features of the system include: data ingestion from multiple sources such as files, databases, and external APIs; data processing and transformation capabilities to handle diverse data formats; data storage in a scalable and secure repository; data analysis and visualization tools to provide insights and trends; and data access controls to ensure authorized personnel can view, edit, or share data. Additionally, the system will include data quality checks, data backup and recovery mechanisms, and integration with existing systems to enhance overall data management.
- The project features a comprehensive data collection and management system, enabling the gathering and organization of large datasets. Additionally, it includes advanced behavioural pattern analysis and visualization tools, allowing for the identification of trends and patterns within the data. These features can be further broken down into: 
   1. Data ingestion: Automated data collection from various sources.
   2. Data processing: Cleaning, transforming, and storing data for analysis.
   3. Data visualization: Interactive and dynamic visualizations to represent complex data insights.
   4. Pattern recognition: Utilizing machine learning algorithms to identify behavioural patterns.
   5. Predictive analytics: Forecasting future trends and outcomes based on historical data.
   6. Real-time monitoring: Continuous monitoring of data streams to detect anomalies and changes in behavioural patterns.
- The project features a comprehensive data collection and management system, which enables the efficient gathering, storage, and organization of data from various sources. This system provides a solid foundation for the subsequent analysis and modelling phases. The behavioural pattern analysis and visualization tools offer in-depth insights into the trends and patterns present within the collected data, allowing for a better understanding of the underlying dynamics. Furthermore, the integration of machine learning algorithms for predictive modelling enables the system to forecast future trends and make predictions based on the analysed patterns, thereby providing a proactive approach to decision-making. Key features include: 
   - Data preprocessing and cleaning 
   - Data visualization through interactive dashboards 
   - Pattern recognition and anomaly detection 
   - Predictive modelling using supervised and unsupervised learning techniques 
   - Model evaluation and validation 
   - Continuous model updating and refinement
- The project encompasses a comprehensive set of features designed to support data-driven research and analysis. The key features include: Data Collection and Management System, Behavioural Pattern Analysis and Visualization Tools, Machine Learning Algorithms for Predictive Modelling, and Collaboration and Sharing Features for research teams.
## Constraints
- The project's constraints are primarily due to the limited data availability on alien behaviour, which restricts our understanding of their potential actions, social structures, and technological capabilities. This scarcity of information hampers the development of comprehensive and accurate models or predictions regarding alien life forms, their habitats, or how they might interact with Earth or other celestial bodies.
- The project is constrained by limited data availability on alien behaviour, making it challenging to gather sufficient information to draw meaningful conclusions. Furthermore, the complexity of analysing and interpreting alien behavioural patterns adds an extra layer of difficulty, requiring advanced methodologies and expertise to decipher and understand the underlying mechanisms driving their actions.
- The project constraints include limited data availability on alien behaviour, which can hinder the development of accurate models and insights. Additionally, analysing and interpreting alien behavioural patterns pose significant complexity due to the unknowns and unpredictability of their actions. Ensuring data security and privacy for sensitive research information is also crucial to prevent unauthorized access and potential repercussions. These constraints necessitate innovative data collection methods, advanced analytical techniques, and stringent security protocols to protect sensitive information and facilitate meaningful research on alien behaviour.
## Assumptions
- To further develop this concept, let's break down the assumptions regarding alien behavioural patterns. It is assumed that alien life forms, if they exist, would exhibit patterns of behaviour that could be observed and recorded. This could include communication methods, social structures, interactions with their environment, and responses to external stimuli. The ability to track and measure these patterns would rely on advanced technologies, such as hyperspectral imaging, artificial intelligence, and machine learning algorithms. Additionally, it is assumed that these behavioural patterns would be consistent and repetitive, allowing for predictive modeling and analysis. The implications of this assumption are significant, as it would enable the development of strategies for communication, cooperation, or even conflict resolution with alien life forms, should they be encountered.
- It is assumed that alien behavioural patterns can be tracked and measured through various methods, including advanced surveillance technology and data collection tools. Additionally, it is assumed that researchers from various fields, including astrobiology, exopaleontology, and the search for extraterrestrial intelligence (SETI), will participate and contribute to the database, providing valuable insights and information about alien behavioural patterns.
- The project assumes that alien behavioural patterns can be tracked and measured through various methods, including but not limited to observations, experiments, and data analysis. It also assumes that researchers from diverse fields, including astrobiology, physics, and mathematics, will participate and contribute to the database, providing a comprehensive understanding of alien behaviour. Furthermore, the application will be used primarily for research purposes, such as identifying trends, patterns, and anomalies in alien behaviour, with potential applications in fields like exopaleontology, xenolinguistics, and the search for extraterrestrial intelligence (SETI).
## Dependencies
- The project's dependencies include access to reliable data sources on alien behavior, advanced data analytics tools, machine learning frameworks, and a team of experts in astrobiology, exoplanetary science, and artificial intelligence. The current value is based on the assumption that such data sources are available, but in reality, the search for reliable data on alien behavior is an ongoing challenge in the fields of astrobiology and the search for extraterrestrial intelligence (SETI).
- The project's dependencies include the availability of reliable data sources on alien behavior and collaboration with research institutions and scientists. Additionally, other key dependencies are: 
   - Access to advanced technology for data collection and analysis, 
   - Interdisciplinary team of experts in astrobiology, astrophysics, and exopaleontology, 
   - Secure funding for research and development, 
   - Establishment of a network for sharing and verifying findings, 
   - Development of effective protocols for communicating with potential extraterrestrial life forms.
- The project's dependencies include the availability of reliable data sources on alien behaviour, which will enable the development of accurate predictive models. Collaboration with research institutions and scientists is also crucial to ensure access to the latest research and expertise in the field. Furthermore, the development of advanced machine learning algorithms for predictive modelling is necessary to analyse the data and make informed predictions about alien behaviour.
## User Personas
- Dr. Maria Rodriguez is a renowned astrobiologist with a Ph.D. in Astrobiology from a prestigious university. She has over 10 years of experience in the field, with a focus on the search for life beyond Earth. Her expertise includes planetary habitability, biosignatures, and the origins of life. Dr. Rodriguez is a leading researcher in her field and has published numerous papers in top-tier scientific journals. She is also an advocate for STEM education and has participated in several outreach programs to promote the importance of astrobiology and space exploration. As the primary user persona for the project, Dr. Rodriguez's goals and needs will drive the development of the system, ensuring that it meets the requirements of astrobiologists and provides a valuable tool for advancing research in the field.
- The user personas for the project currently include Dr. Maria Rodriguez, an astrobiologist, and Dr. John Lee, an extraterrestrial researcher. Dr. Rodriguez specializes in the study of the origin, evolution, distribution, and future of life in the universe, while Dr. Lee focuses on the search for life beyond Earth and the potential for extraterrestrial life forms. Both personas bring unique perspectives to the project, with Dr. Rodriguez providing expertise in the biological aspects of life in space and Dr. Lee contributing knowledge on the possibility of life existing elsewhere in the universe.
- The user personas for the project include Dr. Maria Rodriguez, an astrobiologist who studies the possibility of life beyond Earth, Dr. John Lee, an extraterrestrial researcher focused on detecting intelligent life elsewhere in the universe, and Dr. Sophia Patel, a behavioural scientist who examines human behaviour in the context of space exploration and potential encounters with extraterrestrial life.
## Conclusion
The Alien Behavioural Change Tracker web application has the potential to revolutionize our understanding of alien behaviour and social development, providing valuable insights for researchers and scientists in the field. With its comprehensive data collection and analysis capabilities, this platform can facilitate groundbreaking discoveries, enabling experts to better comprehend the complexities of alien behaviours, their habitats, and potential interactions with other species. Furthermore, the application's user-friendly interface and collaborative features can foster a community of researchers, promoting the sharing of knowledge and expedite the process of understanding these extraterrestrial entities.


In [9]:
## Input Parameters: Change these values to get different PRD and User Stories
project_description = "A web application for tracking fitness goals and workouts."

## Create an instance of MiniPM AI agent
minipm = MiniPM(project_description)

## Get the Product Requirements Document (PRD)
user_stories = minipm.get_user_stories()

## Render the PRD as markdown
print(user_stories.model_dump_json(indent=4))

Augmenting PRD:  {'title': 'Fitness Tracker Web Application', 'overview': 'The Fitness Tracker Web Application is designed to help users track their fitness goals and workouts, providing a comprehensive platform for monitoring progress and achieving a healthier lifestyle.', 'problem_statement': 'Individuals often struggle to maintain a consistent workout routine and track their fitness progress, leading to a lack of motivation and difficulty in achieving their goals.', 'business_domains': ['Health and Wellness', 'Fitness', 'Sports'], 'proposed_solution': 'A user-friendly web application that allows users to set and track fitness goals, log workouts, and monitor progress through customizable dashboards and analytics.', 'target_audience': 'Fitness enthusiasts, athletes, and individuals seeking to improve their overall health and wellness.', 'features': ['User authentication and profile management', 'Goal setting and tracking', 'Workout logging and tracking', 'Customizable dashboards and 